In [1]:
import open3d as o3d
import numpy as np
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## define func

## loading bottle model (target)

In [2]:
mesh_target = o3d.io.read_triangle_mesh('../dataset/model/art_melon.STL')
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.4, origin=[0.0,0.0,0.0])
o3d.visualization.draw_geometries([mesh_target, mesh_frame])

## converting from triangle mesh to pcd

In [3]:
pcd_target = mesh_target.sample_points_uniformly(number_of_points=10000)
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.4, origin=[0.0,0.0,0.0])
o3d.visualization.draw_geometries([pcd_target, mesh_frame])

## loading point cloud (source)

In [4]:
pcd_source = o3d.io.read_point_cloud('../dataset/d405/d405-1.ply')
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0.0,0.0,0.0])
o3d.visualization.draw_geometries([pcd_source, mesh_frame])

## downsampling

In [5]:
voxel_size = 0.005

# source
pcd_source_dsampled = pcd_source.voxel_down_sample(voxel_size)
#pcd_source_dsampled.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([pcd_source_dsampled])

# target (this process is not needed, specified at converting pcd)
pcd_target_dsampled = pcd_target.voxel_down_sample(voxel_size)
#pcd_target_dsampled.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([pcd_target_dsampled])

In [6]:
pcd_target = pcd_target_dsampled
pcd_source = pcd_source_dsampled

## cleaning source point cloud

In [7]:
# crop pcd with aabb (Note. z-axis looks backward of camera)
min_bound = np.asarray([-1.0, -1.0, -1.0]) # [x_min, y_min, z_min]
max_bound = np.asarray([1.0, 1.0, 1.0]) # [x_max, y_max, z_max]
aabb = o3d.geometry.AxisAlignedBoundingBox(min_bound, max_bound)
pcd_source_cropped = pcd_source.crop(aabb)
o3d.visualization.draw_geometries([pcd_source_cropped, mesh_frame])

In [8]:
# remove table 
plane_model, plane_inlier_indeces = pcd_source_cropped.segment_plane(distance_threshold=0.005, ransac_n=3, num_iterations=500)
pcd_plane = pcd_source_cropped.select_by_index(plane_inlier_indeces)
pcd_plane.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([pcd_source_cropped, pcd_plane])
pcd_source_removed_plane = pcd_source_cropped.select_by_index(plane_inlier_indeces, invert=True)
o3d.visualization.draw_geometries([pcd_source_removed_plane])

In [9]:
# remove outlier
cl, inlier_indeces = pcd_source_removed_plane.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
pcd_source_removed_outlier = pcd_source_removed_plane.select_by_index(inlier_indeces)
o3d.visualization.draw_geometries([pcd_source_removed_outlier])

In [10]:
pcd_source = pcd_source_removed_outlier

## keypoint creation

In [11]:
# extract features from point clouds
def keypoint_and_feature_extraction(pcd: o3d.geometry.PointCloud, voxel_size):
    keypoints = pcd.voxel_down_sample(voxel_size)
    viewpoint = np.array([0, 0, 0], dtype='float64')
    radius_normal = 2.0 * voxel_size
    keypoints.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    keypoints.orient_normals_towards_camera_location(viewpoint)

    radius_feature = 5.0 * voxel_size
    feature = o3d.pipelines.registration.compute_fpfh_feature(
        keypoints,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))

    return keypoints, feature

In [12]:
source_kp, source_feature = keypoint_and_feature_extraction(pcd_source, voxel_size)
target_kp, target_feature = keypoint_and_feature_extraction(pcd_target, voxel_size)

In [13]:
def draw_registration_result(source, target, transformation):
    pcds = list()
    for s in source:
        temp = copy.deepcopy(s)
        pcds.append(temp.transform(transformation))
    pcds += target
    o3d.visualization.draw_geometries(pcds)

In [31]:
source_kp.paint_uniform_color([0, 0, 0])
target_kp.paint_uniform_color([0, 0.5, 0.5])
initial_trans = np.identity(4)
initial_trans[0,3] = -3.0
draw_registration_result([pcd_source, source_kp], [pcd_target, target_kp], initial_trans)

[Open3D INFO]   -- Mouse view control --
[Open3D INFO]     Left button + drag         : Rotate.
[Open3D INFO]     Ctrl + left button + drag  : Translate.
[Open3D INFO]     Wheel button + drag        : Translate.
[Open3D INFO]     Shift + left button + drag : Roll.
[Open3D INFO]     Wheel                      : Zoom in/out.
[Open3D INFO] 
[Open3D INFO]   -- Keyboard view control --
[Open3D INFO]     [/]          : Increase/decrease field of view.
[Open3D INFO]     R            : Reset view point.
[Open3D INFO]     Ctrl/Cmd + C : Copy current view status into the clipboard.
[Open3D INFO]     Ctrl/Cmd + V : Paste view status from clipboard.
[Open3D INFO] 
[Open3D INFO]   -- General control --
[Open3D INFO]     Q, Esc       : Exit window.
[Open3D INFO]     H            : Print help message.
[Open3D INFO]     P, PrtScn    : Take a screen capture.
[Open3D INFO]     D            : Take a depth capture.
[Open3D INFO]     O            : Take a capture of current rendering settings.
[Open3D INFO

In [16]:
# ratio test to remove bad keypoints
np_source_feature = source_feature.data.T
np_target_feature = source_feature.data.T

corrs = o3d.utility.Vector2iVector()
threshold = 0.95
for i, feat in enumerate(np_source_feature):
    distance = np.linalg.norm(np_target_feature - feat, axis=1)
    nearest_idx = np.argmin(distance)
    dist_order = np.argsort(distance)
    ratio = distance[dist_order[0]] - distance[dist_order[1]]
    if ratio < threshold:
        corr = np.array([[i], [nearest_idx]], np.int32)
        corrs.append(corr)

print(f'number of correration set: {len(corrs)}')

number of correration set: 7770


In [37]:
# visualize correspondences
def create_lineset_from_correspondenes(corrs_set, pcd1, pcd2, transformation=np.identity(4)):
    pcd1_tmp = copy.deepcopy(pcd1)
    pcd1_tmp.transform(transformation)
    corrs = np.asarray(corrs_set)
    np_points1 = np.array(pcd1_tmp.points)
    np_points2 = np.array(pcd2.points)
    points = list()
    lines = list()

    print(f'np_point1: {len(np_points1)}, np_point2: {len(np_points2)}')

    for i in range(corrs.shape[0]):
        points.append(np_points1[corrs[i, 0]])
        points.append(np_points2[corrs[i, 1]])
        lines.append([2*i, (2*i)+1])

    colors = [np.random.rand(3) for i in range(len(lines))]
    line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(points),
    lines =o3d.utility.Vector2iVector(lines)
    )

    line_set.colors = o3d.utility.Vector3dVector(colors)
    return line_set

In [38]:
line_set = create_lineset_from_correspondenes(corrs, source_kp, target_kp, initial_trans)
draw_registration_result([pcd_source, source_kp], [pcd_target, target_kp, line_set], initial_trans)

np_point1: 7770, np_point2: 4639


IndexError: index 4639 is out of bounds for axis 0 with size 4639

In [21]:
len(source_kp.points)

7770

In [22]:
len(target_kp.points)

4639

In [39]:
len(corrs.shape)

AttributeError: 'open3d.cuda.pybind.utility.Vector2iVector' object has no attribute 'shape'

## RANSAC pipeline including feature extraction

## ICP pipeline

## show result